## Login y Logout

En esta parte, vamos a crear las vistas y la lógica detrás de los **`login`** y **`logout`**, es decir, que sucede cuando entramos y salimos como usuarios de la app web.

Para comenzar, **`django`** tiene vistas pre-diseñadas que podemos usar, vamos al archivo **`urls.py`** del proyecto **`proyecto_django`**, y escribimos:

```python
from django.contrib.auth import views as auth_views
```
*Y agregamos a la lista **`urlpatterns`**:*
```python
path('login/', auth_views.LoginView.as_view(), name = 'login'),
path('logout/', auth_views.LogoutView.as_view(), name = 'logout')
```

Si ejecutamos el servidor, tendremos un error, **`django`** nos dirá que esta buscando el **`template`** para **`login`** en **`registration/login.html`** y no lo va a encontrar.

Aunque es posible crear un nuevo directorio para estos **`templates`**, vamos a decirle a **`django`** que busque en el directorio donde se encuentra **`register.html`**, así tendremos todo en un mismo directorio.

Para eso vamos a cambiar:

```python
path('login/', auth_views.LoginView.as_view(template_name = 'users/login.html'), name = 'login'),
path('logout/', auth_views.LogoutView.as_view(template_name = 'users/logout.html'), name = 'logout')
```

Ahora vamos a crear esos **`html`**.

Creamos un archivo llamado **`login.html`** en el directorio **`templates/users`**, y agregamos:

```html
{% extends "blog/base.html" %}
{% load crispy_forms_tags %}
{% block contenido %}
    <div class="content-section">
        <form method="POST">
            {% csrf_token %}
            <fieldset class="form-group">
                <legend class="border-bottom mb-4">Log In</legend>
                {{ form|crispy}}
            </fieldset>
            <div class="form-group">
                <button class="btn btn-outline-info" type="submit">Login</button>
            </div>
        </form>
        <div class="border-top" pt-3>
            <small class="text-muted">
                Need an account? <a class="ml-2" href="{% url 'register' %}">Sign Up</a>
            </small>
        </div>
    </div>
{% endblock contenido %}
```

**Nota**: agregar el **`href`** al archivo **`register.html`** (**`{% url 'login' %}`**)

Con esto podemos visualizar la página de **`login`**.

**Ejecutamos el servidor para ver los cambios.**

Si escribimos un usuario que no existe o una contraseña incorrecta, la página envia un mensaje de error.

Si escribimos un usuario que si existe con la contraseña correcta, la app nos mandará automaticamente a una página de perfil (**`localhost:8000/account/profile`**), este es el comportamiento por defecto, ya que no hemos creado ningún **`url`** ni **`templates`** para perfiles.

Para este ejemplo, vamos a decirle a **`django`** que cada vez que un usuario haga **`login`** lo rediriga a **`home`**.

Primero vamos a **`settings.py`** y en la última linea del código agregamos:
```python
LOGIN_REDIRECT_URL = 'blog-home'
```

Solo con esta linea de código, al volver a hacer **`login`** ya nos redirige a **`home`**.

Con la página de **`login`** finalizada, vamos a cambiar lo siguiente del **`views.py`**: 

*Cambiamos:*
```python
messages.success(request, f'Account created for {username}')
return redirect('blog-home')
```

*Por:*
```python
messages.success(request, f'Your account has been created! You are now able to log in.')
return redirect('login')
```

Ahora nos redirigirá a la página de **`login`** cuando nos hayamos registrado.


Hagamos lo mismo con la página de **`logout`**:
```html
{% extends "blog/base.html" %}
{% block contenido %}
    <h2>You have been logged out.</h2>
    <div class="border-top" pt-3>
        <small class="text-muted">
            <a href="{% url 'login' %}">Log In Again</a>
        </small>
    </div>
{% endblock contenido %}
```
Aquí solo vamos a imprimir que se ha hecho **`logout`** y dar la opción de hacer **`login`** redirigiendo a la pagina de **`login`**.


Para finalizar, la barra de navegación siempre muestra los botones de **`LogIn`** y **`Register`** aun cuando el usuario ya ha hecho **`login`**. Cambiemos esto para que muestre algo con más sentido.

Vamos a **`base.html`** y lo primero es agregar los **`url`** a **`LogIn`** y **`Register`**, es decir:

*Cambiamos:*
```html
<a class="nav-item nav-link" href="#">Login</a>
<a class="nav-item nav-link" href="#">Register</a>
```
*Por:*
```html
<a class="nav-item nav-link" href="{% url 'login' %}">Login</a>
<a class="nav-item nav-link" href="{% url 'register' %}">Register</a>
```

Y vamos con la parte del condicional, vamos a agregar:
```html
<div class="navbar-nav">
    {% if user.is_authenticated %}
        <a class="nav-item nav-link" href="{% url 'logout' %}">Logout</a>
    {% else %}
        <a class="nav-item nav-link" href="{% url 'login' %}">Login</a>
        <a class="nav-item nav-link" href="{% url 'register' %}">Register</a>
    {% endif %}
```

## Profile (Perfil)

Comencemos con el **`view`**:

```python
def profile(request):
    return render(request, 'users/profile.html')
```

Y vamos a crear un **`template`** que se llame **`profile.html`**:

```html
{% extends "blog/base.html" %}
{% load crispy_forms_tags %}
{% block contenido %}
    <h1>{{ user.username }}</h1>
{% endblock contenido %}
```
Vamos a crear la ruta en **`urls.py`** de **`django_proyect`**, agregamos a la lista **`urlpatterns`**:

```python
path('profile/', user_views.profile, name = 'profile')
```
Y por último vamos a agregar el botón de **`Profile`** a la barra de navegación, vamos a **`base.html`** y agregamos a la parte del condicional **`if`**:
```html
<a class="nav-item nav-link" href="{% url 'profile' %}">Profile</a>
```

**Ejecutamos el servidor para ver los resultados.**

Vemos que nos muestra el nombre del usuario en **`/profile`**, hagamos **`logout`** y vamos a intentar entrar a la página de **`/profile`**...

Ahora la página no nos muestra nada, pero tenemos "acceso" a ella. Para que esto no pueda suceder, vamos a modificar la función **`profile`** de **`views.py`** usando un **decorador**:

```python
from django.contrib.auth.decorators import login_required

@login_required
def profile(request):
    return render(request, 'users/profile.html')
```

Con este decorador solucionamos el problema, pero aparece otro, al intentar entrar a esta página sin haber hecho **`login`** nos redirige a una página que no existe. Vamos a hacer que nos rediriga a **`login`**.

Vamos a la **`settings.py`** y vamos a la ultima linea y escribimos:
```python
LOGIN_URL = 'login'
```

**Ejecutamos el servidor para ver los resultados.**